# Import packages

In [ ]:
#update earth engine if necessary
# !pip install earthengine-api --upgrade

In [ ]:
#import all necessary packages
import ee
from Py6S import *
import datetime
import math
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from atmospheric import Atmospheric

ee.Authenticate()
ee.Initialize(project='ee-curuai') #change project name here

Enter verification code: 4/1AfJohXnWTZFRf0Pd6RUYKQktR161BlNByuk9m0Aw7mFYJGoCWwYS_UV6orU

Successfully saved authorization token.


In [ ]:
#function to insert a property with date of the point - Gee date format
def insert_date (feat):
    return feat.set('system:time_start',ee.Date.parse('YYYY-MM-dd HH:mm:ss',feat.get('DATE')))

In [ ]:
#define the mission number
mission = 'I'
n = 1

In [ ]:
#filter database by mission and insert a date to the points used
dados =ee.FeatureCollection('projects/ee-curuai/assets/curuai_points').filter(ee.Filter.eq('MISSION',mission)).map(insert_date)
print(dados.getInfo())

In [ ]:
#set the geometry roi
region = dados.geometry() 
geom = region

In [ ]:
# get the first and last dates os sample collection in the mission and extend the range by 16 days (Landsat interval between images)
startDate = ee.Date(dados.sort('system:time_start').first().get('system:time_start')).advance(-16, 'day')
endDate = ee.Date(dados.sort('system:time_start',False).first().get('system:time_start')).advance(16, 'day')

print('Data inicial de coleta: ',startDate.format().getInfo())
print('Data final de coleta: ',endDate.format().getInfo())

In [ ]:
#define a maximum cloud cover of 90% 
max_cloud_cover = 90

# Register images function

In [ ]:
def register_images(imgToregister):
    #reference images are Sentinel 2 of orbit T21MXT for displacing Landsat 7 and 8 of orbit 228061 and 228062
    #reference images are Sentinel 2 of orbit T21MYT for displacing Landsat 7 and 8 of orbit 227062 
    ref = ee.Algorithms.If(ee.Algorithms.IsEqual(imgToregister.getNumber('WRS_PATH'), 227),
                           ee.Image("COPERNICUS/S2_HARMONIZED/20160405T141300_20160408T012329_T21MYT"), #change the refence images here
                                            ee.Image("COPERNICUS/S2_HARMONIZED/20160405T141300_20160408T012329_T21MXT")) #change the refence images here
        
    #Use bicubic resampling during registration.
    refrig = ee.Image(ref).resample('bicubic')
    imgToregisterrig = imgToregister.resample('bicubic')
    
    #Choose to register using only the Red band.
    refRedBand = refrig.select('B4')
    imgToregisterRedBand = imgToregisterrig.select('B4')
    
    #Determine the displacement by matching only the 'R' bands.
    displacement = imgToregisterRedBand.displacement(**{
        'referenceImage': refRedBand,
        'maxOffset': 50.0,
    })
    #Use the computed displacement to register all original bands.
    registered = imgToregisterrig.displace(displacement)
    
    return registered

# Py6S atm Correction

## Landsat 7

In [ ]:
def ld7_corr(img):
    img = ee.Image(img)
    info = img.getInfo()['properties']
    scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
    solar_elv = img.getInfo()['properties']['SUN_ELEVATION']
    solar_z = ee.Number(90).subtract(solar_elv).getInfo()
    h2o = Atmospheric.water(geom,img.date()).getInfo()
    o3 = Atmospheric.ozone(geom,img.date()).getInfo()
    aot = Atmospheric.aerosol(geom,img.date()).getInfo()
    SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
    km = alt/1000 # i.e. Py6S uses units of kilometers
    # Instantiate
    s = SixS()
    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AeroProfile.Continental
    s.aot550 = aot
    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    #s.geometry.solar_z = solar_z        # solar zenith angle
    s.geometry.solar_z = solar_z # solar zenith angle
    s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
    s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(km)
    #Mission specific for atmospheric correction
    #https://github.com/samsammurphy/ee-atmcorr-timeseries/blob/master/atmcorr/mission_specifics.py
    def spectralResponseFunction(bandname):        
        bandSelect = {
            'B1':PredefinedWavelengths.LANDSAT_ETM_B1,
            'B2':PredefinedWavelengths.LANDSAT_ETM_B2,
            'B3':PredefinedWavelengths.LANDSAT_ETM_B3,
            'B4':PredefinedWavelengths.LANDSAT_ETM_B4,
            'B5':PredefinedWavelengths.LANDSAT_ETM_B5,
            'B7':PredefinedWavelengths.LANDSAT_ETM_B7,
                    }
        return Wavelength(bandSelect[bandname])
    def toa_to_rad(bandname):
        ESUN_L7 = [1997, 1812, 1533, 1039, 230.8, 84.9] # PAN =  1362 (removed to match Py6S)
        ESUN_BAND = {
            'B1':ESUN_L7[0],
            'B2':ESUN_L7[1],
            'B3':ESUN_L7[2],
            'B4':ESUN_L7[3],
            'B5':ESUN_L7[4],            
            'B7':ESUN_L7[5],
           }
        solar_angle_correction = math.cos(math.radians(solar_z))
        # Earth-Sun distance (from day of year)
        doy = scene_date.timetuple().tm_yday
        d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
        # conversion factor
        multiplier = ESUN_BAND[bandname]*solar_angle_correction/(math.pi*d**2)
        # at-sensor radiance
        rad = img.select(bandname).multiply(multiplier)
        return rad
    def surface_reflectance(bandname):
        # run 6S for this waveband
        s.wavelength = spectralResponseFunction(bandname)
        s.run()

        # extract 6S outputs
        Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
        Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
        Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
        absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
        scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
        tau2 = absorb*scatter                                #total transmissivity

        # radiance to surface reflectance
        rad = toa_to_rad(bandname)
        ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
        return ref
    
    
    blue = ee.Image(surface_reflectance('B1'))
    green = ee.Image(surface_reflectance('B2'))
    red = ee.Image(surface_reflectance('B3'))
    nir = ee.Image(surface_reflectance('B4'))
    swir1 = ee.Image(surface_reflectance('B5'))
    swir2 = ee.Image(surface_reflectance('B7'))
    
    ref = img.select('QA_PIXEL').addBands(blue).addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2)
    
    dateString = scene_date.strftime("%Y-%m-%d")
    ref = ref.copyProperties(img).set({              
                  'AC_date':dateString,
                  'AC_aerosol_optical_thickness':aot,
                  'AC_water_vapour':h2o,
                  'AC_version':'py6S',
                  'AC_ozone':o3,
                  'MISSION':mission})#change properties to set here
    

    # define YOUR assetID 
    # export

    fname = ee.String(img.get('system:index')).getInfo()
    export = ee.batch.Export.image.toAsset(\
        image=ee.Image(ref),
        description= 'L7_BOA_'+fname,
        assetId = 'projects/ee-curuai/assets/Py6S/mission'+str(n)+'/'+'BOA_'+fname,#change the properties to add in images here
        region = region.buffer(50).bounds(),
        scale = 30,
        maxPixels = 1e13)

    # # uncomment to run the export
    export.start()
    print('exporting ' +fname + '--->done')
    return ref

In [ ]:
#filter collection by data, roi and maximum cloud cover. Then apply registering function
L7_col = ee.ImageCollection("LANDSAT/LE07/C02/T1_TOA")\
            .filterBounds(region).filterDate(startDate,endDate)\
            .filter(ee.Filter.lt('CLOUD_COVER',max_cloud_cover))\
            .map(register_images)

In [ ]:
print(L7_col.size().getInfo())

25


In [ ]:
col_length = L7_col.size().getInfo()
#print(col_length)

for i in range(0,col_length):
    #print(i)
    list = L7_col.toList(col_length)
    img = ee.Image(list.get(i))
    ld7_corr(img)

## Landsat 8

In [ ]:
def ld8_corr(img):
    img = ee.Image(img)
    info = img.getInfo()['properties']
    scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
    solar_elv = img.getInfo()['properties']['SUN_ELEVATION']
    solar_z = ee.Number(90).subtract(solar_elv).getInfo()
    h2o = Atmospheric.water(geom,img.date()).getInfo()
    o3 = Atmospheric.ozone(geom,img.date()).getInfo()
    aot = Atmospheric.aerosol(geom,img.date()).getInfo()
    SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
    km = alt/1000 # i.e. Py6S uses units of kilometers
    # Instantiate
    s = SixS()
    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AeroProfile.Continental
    s.aot550 = aot
    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    #s.geometry.solar_z = solar_z        # solar zenith angle
    s.geometry.solar_z = solar_z # solar zenith angle
    s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
    s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(km)
    def spectralResponseFunction(bandname):        
        bandSelect = {
            'B1':PredefinedWavelengths.LANDSAT_OLI_B1,
            'B2':PredefinedWavelengths.LANDSAT_OLI_B2,
            'B3':PredefinedWavelengths.LANDSAT_OLI_B3,
            'B4':PredefinedWavelengths.LANDSAT_OLI_B4,
            'B5':PredefinedWavelengths.LANDSAT_OLI_B5,
            'B6':PredefinedWavelengths.LANDSAT_OLI_B6,
            'B7':PredefinedWavelengths.LANDSAT_OLI_B7,
            'B8':PredefinedWavelengths.LANDSAT_OLI_B8,
            'B9':PredefinedWavelengths.LANDSAT_OLI_B9,
                    }
        return Wavelength(bandSelect[bandname])
    def toa_to_rad(bandname):
        ESUN_L8 = [1895.33, 2004.57, 1820.75, 1549.49, 951.76, 247.55, 85.46, 1723.8, 366.97]
        ESUN_BAND = {
            'B1':ESUN_L8[0],
            'B2':ESUN_L8[1],
            'B3':ESUN_L8[2],
            'B4':ESUN_L8[3],
            'B5':ESUN_L8[4],
            'B6':ESUN_L8[5],
            'B7':ESUN_L8[6],
            'B8':ESUN_L8[7],
            'B9':ESUN_L8[8],
            }
        solar_angle_correction = math.cos(math.radians(solar_z))
        # Earth-Sun distance (from day of year)
        doy = scene_date.timetuple().tm_yday
        d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
        # conversion factor
        multiplier = ESUN_BAND[bandname]*solar_angle_correction/(math.pi*d**2)
        # at-sensor radiance
        rad = img.select(bandname).multiply(multiplier)
        return rad
    def surface_reflectance(bandname):
        # run 6S for this waveband
        s.wavelength = spectralResponseFunction(bandname)
        s.run()

        # extract 6S outputs
        Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
        Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
        Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
        absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
        scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
        tau2 = absorb*scatter                                #total transmissivity

        # radiance to surface reflectance
        rad = toa_to_rad(bandname)
        ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
        return ref
    
    ca  = surface_reflectance('B1')
    blue = surface_reflectance('B2')
    green = surface_reflectance('B3')
    red = surface_reflectance('B4')
    nir = surface_reflectance('B5')
    swir1 = surface_reflectance('B6')
    swir2 = surface_reflectance('B7')
    
    ref = img.select('QA_PIXEL').addBands(ca).addBands(blue).addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2)
    
    dateString = scene_date.strftime("%Y-%m-%d")
    ref = ref.copyProperties(img).set({              
                  'AC_date':dateString,
                  'AC_aerosol_optical_thickness':aot,
                  'AC_water_vapour':h2o,
                  'AC_version':'py6S',
                  'AC_ozone':o3,
                  'MISSION':mission})#change the properties to add in images here
    

    
    fname = ee.String(img.get('system:index')).getInfo()
    export = ee.batch.Export.image.toAsset(\
        image=ee.Image(ref),
        description= 'L8_BOA_'+fname,
        assetId = 'projects/ee-curuai/assets/Py6S/mission'+str(n)+'/'+'BOA_'+fname, #change exporting path here
        region = region.buffer(50).bounds(),
        scale = 30,
        maxPixels = 1e13)

    # # uncomment to run the export
    export.start()
    print('exporting ' +fname + '--->done')
    return ref

In [ ]:
#filter collection by data, roi and maximum cloud cover. Then apply registering function
L8_col = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")\
            .filterBounds(region).filterDate(startDate,endDate)\
            .filter(ee.Filter.lt('CLOUD_COVER',max_cloud_cover))\
            .map(register_images)

In [ ]:
print(L8_col.size().getInfo())

In [ ]:
col_length = L8_col.size().getInfo()
#print(col_length)

for i in range(0,col_length):
    #print(i)
    list = L8_col.toList(col_length)
    img = ee.Image(list.get(i))
    ld8_corr(img)

## Sentinel 2

In [ ]:
def s2_corr(img):
    img = ee.Image(img)
    info = img.getInfo()['properties']
    scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
    solar_z = img.getInfo()['properties']['MEAN_SOLAR_ZENITH_ANGLE']
    h2o = Atmospheric.water(geom,img.date()).getInfo()
    o3 = Atmospheric.ozone(geom,img.date()).getInfo()
    aot = Atmospheric.aerosol(geom,img.date()).getInfo()
    SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
    km = alt/1000 # i.e. Py6S uses units of kilometers
    # Instantiate
    s = SixS()
    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AeroProfile.Continental
    s.aot550 = aot
    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    s.geometry.solar_z = solar_z # solar zenith angle
    s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
    s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(km)
    def spectralResponseFunction(bandname):        
        
        bandSelect = {
            'B1':PredefinedWavelengths.S2A_MSI_01,
            'B2':PredefinedWavelengths.S2A_MSI_02,
            'B3':PredefinedWavelengths.S2A_MSI_03,
            'B4':PredefinedWavelengths.S2A_MSI_04,
            'B5':PredefinedWavelengths.S2A_MSI_05,
            'B6':PredefinedWavelengths.S2A_MSI_06,
            'B7':PredefinedWavelengths.S2A_MSI_07,
            'B8':PredefinedWavelengths.S2A_MSI_08,
            'B8A':PredefinedWavelengths.S2A_MSI_8A,
            'B9':PredefinedWavelengths.S2A_MSI_09,
            'B10':PredefinedWavelengths.S2A_MSI_10,
            'B11':PredefinedWavelengths.S2A_MSI_11,
            'B12':PredefinedWavelengths.S2A_MSI_12,
                        }
        return Wavelength(bandSelect[bandname])
    def toa_to_rad(bandname):
        ESUN = info['SOLAR_IRRADIANCE_'+bandname]
        solar_angle_correction = math.cos(math.radians(solar_z))
        # Earth-Sun distance (from day of year)
        doy = scene_date.timetuple().tm_yday
        d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
        # conversion factor
        multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
        # at-sensor radiance
        rad = img.select(bandname).multiply(multiplier)
        return rad
    def surface_reflectance(bandname):
        # run 6S for this waveband
        s.wavelength = spectralResponseFunction(bandname)
        s.run()

        # extract 6S outputs
        Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
        Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
        Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
        absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
        scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
        tau2 = absorb*scatter                                #total transmissivity

        # radiance to surface reflectance
        rad = toa_to_rad(bandname)
        ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
        return ref
    
    ca  = surface_reflectance('B1')
    blue = surface_reflectance('B2')
    green = surface_reflectance('B3')
    red = surface_reflectance('B4')
    nir = surface_reflectance('B8')
    swir1 = surface_reflectance('B11')
    swir2 = surface_reflectance('B12')
    
    ref = img.select('QA60').addBands(ca).addBands(blue).addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2)
    
    dateString = scene_date.strftime("%Y-%m-%d")
    ref = ref.copyProperties(img).set({              
                  'AC_date':dateString,
                  'AC_aerosol_optical_thickness':aot,
                  'AC_water_vapour':h2o,
                  'AC_version':'py6S',
                  'AC_ozone':o3,
                  'MISSION':mission}) #change properties to set here
    

    # define YOUR assetID 
    # in my case it was something like this..
    #assetID = 'users/samsammurphy/shared/sentinel2/6S/ESRIN_'+dateString
    #assetID = 'users/ndminhhus/eLEAF/nt/s2_SIAC/'+fname,
    # # export
    fname = ee.String(img.get('system:index')).getInfo()
    export = ee.batch.Export.image.toAsset(\
        image=ee.Image(ref),
        description= 'S2_BOA_'+fname,
        assetId = 'projects/ee-curuai/assets/Py6S/mission'+str(n)+'/'+'BOA_'+fname,#change the properties to add in images here
        region = region.buffer(50).bounds(),
        scale = 10,
        maxPixels = 1e13)

    # # uncomment to run the export
    export.start()
    print('exporting ' +fname + '--->done')
    return ref

In [ ]:
#filter collection by data, roi and maximum cloud cover. Then apply registering function
S2_col = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")\
            .filterBounds(region).filterDate(startDate,endDate)\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',max_cloud_cover))\
            .map(register_images)



In [ ]:
print(S2_col.size().getInfo())

In [ ]:
col_length = S2_col.size().getInfo()
#print(col_length)

for i in range(0,col_length):
    #print(i)
    list = S2_col.toList(col_length)
    img = ee.Image(list.get(i))
    s2_corr(img)